In [1]:
import numpy as np

## LM Head Forwording

LM head คือ ตัวแปลงผลลัพธ์ที่ได้จาก transformer block (ที่ให้ output ออกมาเป็น hidden states ของแต่ละตำแหน่งในประโยค) 

Mathemetic definition

$$Logits = h_{last} \cdot W_{vocab} + b_{vocab}$$

Where 
* $h_{last} \in R^{(d_{model}, )}$
* $W_{vocab} \in R^{(d_{model} \times vocab\_size)}$
* $b_{vocab} \in R^{(vocab\_size,)}$ 
* $logits \in R^{(vocab\_size,)}$ 

$h_{last}$ คือ hidden state ของ token ตัวสุดท้ายที่ได้จาก stack ของ transformer 

In [2]:
d_model = 6
vocab_size = 8
V = vocab_size
T = 4


In [3]:
W_vocab = np.random.randn(d_model, V)
b_vocab = np.random.randn(vocab_size)
h = np.random.rand(T, d_model)

print(h)

[[0.44715079 0.9865368  0.71745372 0.05371566 0.09537326 0.02642603]
 [0.86172301 0.09298135 0.47184439 0.45144861 0.47076121 0.67899689]
 [0.69668911 0.97511833 0.24501172 0.7758092  0.61267186 0.58019618]
 [0.78074696 0.86876477 0.55813452 0.2556323  0.47588194 0.68764426]]


In [4]:
logits = h @ W_vocab + b_vocab
print(logits)

[[-1.52983508  0.70310997  0.5364591   2.50692033  1.34728201 -2.01254737
   1.9890251  -0.25359821]
 [-0.83900195  0.58196978  2.16097193  0.83057337 -0.07052125 -1.39904382
   1.05056475 -1.17870034]
 [-1.44771155 -0.31709931  2.03876204  1.55747901  1.60176219 -0.53427066
   0.26109167 -1.55538583]
 [-2.01387605  0.48272571  2.26576388  2.21219624  1.0512282  -1.58177816
   1.21262073 -0.81688886]]


### In case of many batch

In [5]:
B = 2 # batch size

In [6]:
h = np.random.rand(B, T, d_model)
print (h.shape)

(2, 4, 6)


In [7]:
W_vocab.shape

(6, 8)

In [8]:
h[:, -1, :].shape

(2, 6)

In [9]:
logits = (h[:, -1, :] @ W_vocab) + b_vocab
print("logits => \n")
print(logits)

logits => 

[[-0.18572803 -0.32837398  2.32165547  0.35092136  1.41241838  0.07576595
   0.36084868 -1.19040935]
 [-0.86941022  0.08087081  2.14004662  1.27632003  1.34176311 -0.74419749
   1.11865672 -0.99444997]]


## Back propagation of LM head

from the last chapter (loss And Softmax)

เราทราบว่า

$$\frac {\partial \mathcal {L}} {\partial p} \cdot \frac{\partial p} {\partial Z} = \frac {\partial \mathcal {L}} {\partial Z} \in R^{B \times T \times V}$$

หรือมีมิติเท่ากับ logits

เมื่อ 
* $\mathcal {L}$ คือ loss ของ model
* $p$ คือ ความน่าจะเป็นของแต่ละคำที่จะออกเป็น token ถัดไป 
* $Z$ คือ Logits ที่ได้จากการคำนวณด้วย LM Head

In [10]:
dZ = np.random.rand(B, T, V)
print("dZ =>", dZ.shape)

dZ => (2, 4, 8)


### Start from math
---

#### เมื่อต้องการปรับค่า $W_{vocab}$ เพื่อให้ loss มีค่าลดลงจะต้องปรับโดยการ : 
$$ W^{new}_{vocab} = W^{old}_{vocab} - \alpha \frac{\partial \mathcal {L}} {\partial W_{vocab}}$$

จาก Chain rule : 

$$\frac{\partial \mathcal {L}} {\partial W_{vocab}} = \frac{\partial \mathcal {L}} {\partial p} \cdot \frac{\partial p} {\partial Z} \cdot \frac{\partial Z} {\partial W_{vocab}}$$

ซึ่ง 

$$ \frac{\partial Z} {\partial W_{vocab}} = h_{last\ token}$$

ได้ว่า

$$ W^{new}_{vocab} = W^{old}_{vocab} - \alpha \frac{\partial \mathcal {L} } {\partial Z} \cdot h_{last\ token}$$

เมื่อ $\alpha$ คือ learning rate

#### เมื่อต้องการปรับค่า $b_{vocab}$ เพื่อให้ loss มีค่าลดลงจะต้องปรับโดยการ : 

$$ b^{new}_{vocab} = b^{old}_{vocab} - \alpha \frac{\partial \mathcal {L}} {\partial b_{vocab}}$$

จาก Chain rule : 

$$\frac{\partial \mathcal {L}} {\partial b_{vocab}} = \frac{\partial \mathcal {L}} {\partial p} \cdot \frac{\partial p} {\partial Z} \cdot \frac{\partial Z} {\partial b}$$

ซึ่ง 

$$ \frac{\partial Z} {\partial b_{vocab}} = 1$$

ได้ว่า

$$ b^{new}_{vocab} = b^{old}_{vocab} - \alpha \frac{\partial \mathcal {L} } {\partial Z}$$

แต่โดยปรกติแล้วเราจะไม่ทำตามวิธีข้างต้นเนื่องจากกิน Resource เนื่องจากคิดจะทายแค่ token ถัดไปตัวเดียว 
แต่จะคิดทุกตำแหน่งแทน

ตอน train เราทำ teacher forcing : ทุกตำแหน่งใน sequence มี target ของตัวเอง จาก (forward side) &rarr;
* $h \in \mathbb{R} ^ {B \times T \times d}$
* $logits = h\ @\ W_{vocab} + b_{vocab} \in \mathbb{R} ^ {B \times T \times V}$



Backward propagation ของ LM Head :
มีลำดับการไหลย้อนกลับดังนี้

**loss &rarr; logits (dZ) &rarr; W, b &rarr; hidden (dh)**

เมื่อ forward คือ : 

$$Z_{b, t, V} = h_{b, t, d} W + b$$

1. เราต้องมี gradient of logits ก่อน

ถ้า loss เป็น softmax + cross entropy แบบมาตรฐาน เราจะได้ว่า

$$\frac {\partial \mathcal{L}}{\partial Z} = dZ = \frac {P - Y} {N}$$

Where
* $P$ is softmax(logits) $P \in \mathbb {R} ^ {B, T, V}$
* $Y$ is one-hot of target $Y \in \mathbb {R} ^ {B, T, V}$
* $N$ is token count in loss after mask

2. **from $dZ$ to $dW, db$**

$$\frac {\partial \mathcal {L}} {\partial W} = dW = \sum_{b, t} h_{b, t}^\intercal dZ_{b, t}$$

$$\frac {\partial \mathcal {L}} {\partial b} = db = \sum_{b, t} dZ_{b, t}$$

3. **find $dh$**

from chain rule :
$$ \frac {\partial \mathcal {L}} {\partial h} = \frac {\partial \mathcal {L}} {\partial Z} \cdot \frac {\partial Z} {\partial h} = dh$$

$$dh_{b, t} = dZ_{b, t} W^\intercal$$

### Code demo
---

In [11]:
alpha = 0.1


เนื่องจากการดำเนินการ 3 มิติทำได้ยาก วิธีที่น่าจะดีกว่าคือการรวมมิติของ batch และ token เข้าด้วยกัน (กลายเป็น 2 มิติโดยการ)

In [15]:
print("shape of h before reshape: ", h.shape)
h = h.reshape(B * T, d_model)
print("shape of h after reshape: ", h.shape)

shape of h before reshape:  (2, 4, 6)
shape of h after reshape:  (8, 6)


In [16]:
print("shape of dZ before reshape: ", dZ.shape)
dZ = dZ.reshape(B * T, V)
print("shape of dZ after reshape : ", dZ.shape)

shape of dZ before reshape:  (2, 4, 8)
shape of dZ after reshape :  (8, 8)


In [26]:
dW = h.T @ dZ
print("now shape of dW will be (d, V) => ", dW.shape)
print("must be equal to shape of W_vocab =>", W_vocab.shape)

now shape of dW will be (d, V) =>  (6, 8)
must be equal to shape of W_vocab => (6, 8)


In [27]:
db = np.sum(dZ, axis=0)
print("now shape of db will be (V, ) => ", db.shape)
print("same as dW, db shape must be the same as b_vocab", b_vocab.shape)

now shape of db will be (V, ) =>  (8,)
same as dW, db shape must be the same as b_vocab (8,)


In [24]:
dh = dZ @ W_vocab.T
print("current shape of dh will be (B * T, d_model) => ", dh.shape)

dh = dh.reshape(B, T, d_model)
print("then reshape dh back to (B, T, d_model) => ", dh.shape)

current shape of dh will be (B * T, d_model) =>  (8, 6)
then reshape dh back to (B, T, d_model) =>  (2, 4, 6)


In [25]:
dh

array([[[ 0.65571554,  2.00689945,  2.088832  , -1.5798351 ,
          3.0144384 , -0.63085032],
        [ 1.10440019, -0.39190213,  2.73527592,  0.13243104,
          0.65048795,  0.08091257],
        [ 0.83571842, -0.08091087,  1.88466722, -0.46636526,
          2.33031572, -0.29214401],
        [ 0.18582657,  2.35503702,  2.38039893, -2.47268695,
          0.61815483, -0.36993217]],

       [[ 0.05225005,  1.49911943,  0.75467931, -0.73489231,
          2.65561534, -0.64431432],
        [ 1.43965782,  1.05678308,  1.21509206, -0.8894434 ,
          2.98186949,  0.01973831],
        [ 0.11951202, -0.35594102,  3.15787593,  0.09920057,
          0.7350311 , -1.16988751],
        [-0.59049026,  0.84869468,  3.29997511, -0.48238096,
          1.30174718, -1.33410353]]])

In [14]:
print(h)

[[[0.6869709  0.47320153 0.7736089  0.02466135 0.57355361 0.5485553 ]
  [0.59153033 0.89930792 0.67462012 0.15341159 0.04037051 0.83801591]
  [0.57431326 0.52027146 0.92902884 0.043615   0.0633114  0.49497618]
  [0.64792975 0.20082103 0.45274823 0.64216354 0.98124653 0.32218366]]

 [[0.30989129 0.35783533 0.6943934  0.18722026 0.53930606 0.66635193]
  [0.46608503 0.89348164 0.98441282 0.56197312 0.50989917 0.73589451]
  [0.81225849 0.04727294 0.60832117 0.58246372 0.58723661 0.33844538]
  [0.71700372 0.61109656 0.61459111 0.55671557 0.68823551 0.36489415]]]


h2 = h.copy()
h2.resize(h.shape[0] * h.shape[1], h.shape[2])
print(h2)

h2.resize(h.shape[0], h.shape[1], h.shape[2])



print(h2)

In [ ]:
h.T @ dZ

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 4 is different from 2)

In [ ]:
W_vocab.shape

(6, 8)

In [ ]:
W_vocab_new = W_vocab - alpha * 